In [21]:
import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast
import time

In [22]:
df = pd.read_csv("train.csv")
df

,untagged_sentence,tagged_sentence
0,"['The', 'jury', 'further', 'said', 'in', 'term...","[('The', 'AT'), ('jury', 'NN'), ('further', 'R..."
1,"['The', 'September-October', 'term', 'jury', '...","[('The', 'AT'), ('September-October', 'NP'), (..."
2,"['``', 'Only', 'a', 'relative', 'handful', 'of...","[('``', '``'), ('Only', 'RB'), ('a', 'AT'), ('..."
3,"['The', 'jury', 'said', 'it', 'did', 'find', '...","[('The', 'AT'), ('jury', 'NN'), ('said', 'VB')..."
4,"['It', 'recommended', 'that', 'Fulton', 'legis...","[('It', 'PP'), ('recommended', 'VB'), ('that',..."
...,...,...
47335,"['S.', 'J.', 'Perelman']","[('S.', 'NP'), ('J.', 'NP'), ('Perelman', 'NP')]"
47336,"['revulsion', 'in', 'the', 'desert']","[('revulsion', 'NN'), ('in', 'IN'), ('the', 'A..."
47337,"['the', 'doors', 'of', 'the', 'D', 'train', 's...","[('the', 'AT'), ('doors', 'NN'), ('of', 'IN'),..."
47338,"['She', 'was', 'a', 'living', 'doll', 'and', '...","[('She', 'PP'), ('was', 'BE'), ('a', 'AT'), ('..."


In [23]:
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence']))

47340it [00:05, 8146.16it/s] 


In [136]:
x = []
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j][1] not in x:
            x.append(data[i][j][1])
print(len(x))

49


In [137]:
count_of_tags = {string: 0 for i, string in enumerate(x)}
for i in range(len(data)):
    for j in range(len(data[i])):
        count_of_tags[data[i][j][1]] += 1

In [159]:
vocab = set()
for row in data:
    for word in row:
        vocab.add(word[0])
vocab = list(vocab)
print(len(vocab))
 
keys = [i for i in range(len(vocab))]
vocab_dict = dict(zip(vocab , keys))

51208


In [139]:
all_tags = [ele for ele in x]
values = [i for i in range(49)]
tags_to_index = {tag: value for tag , value in zip(all_tags,values)}

In [141]:
x_dim , y_dim = len(vocab) , 50
data_type = np.float32
bigrams = np.zeros((x_dim, y_dim), dtype=data_type)

In [143]:
x_dim, y_dim, z_dim = len(vocab), 50, 50
data_type = np.float32
trigrams = np.zeros((x_dim, y_dim, z_dim), dtype=data_type) 

In [144]:
for i in range(len(data)):
    for j in range(len(data[i])):
        if j ==0:
            bigrams[vocab_dict[data[i][j][0]]][0] += 1
        else:
            bigrams[vocab_dict[data[i][j][0]]][tags_to_index[data[i][j][1]]] += 1

In [145]:
#trigram_count = [[[0 for _ in range(49)] for _ in range(49)] for _ in range(len(vocab))]
for i in range(len(data)):
    for j in range(len(data[i])):
        if j == 0:
            trigrams[vocab_dict[data[i][j][0]]][tags_to_index[data[i][j][1]]][0] += 1
        elif j == len(data[i])-1:
            trigrams[vocab_dict[data[i][j][0]]][49][tags_to_index[data[i][j][1]]] += 1
        else:
             trigrams[vocab_dict[data[i][j][0]]][tags_to_index[data[i][j][1]]][tags_to_index[data[i][j-1][1]]] += 1

In [146]:
matrix = np.zeros((x_dim, y_dim, z_dim)) 

In [151]:
for i in range(len(data)):
    for j in range(len(data[i])):
        curr_word = vocab_dict[data[i][j][0]]
        curr_tag = tags_to_index[data[i][j][1]]
        prev_tag = tags_to_index[data[i][j-1][1]]
        if j==0:
            prev_tag = 0
        if j == len(data[i]) - 1:
            curr_tag = 49
            prev_tag = tags_to_index[data[i][j][1]]
        matrix[curr_word][curr_tag][prev_tag] = (trigrams[curr_word][curr_tag][prev_tag] + 1) / (bigrams[curr_word][prev_tag] + len(vocab))

In [106]:
test = pd.read_csv("test_small.csv")
test

,Unnamed: 0,id,untagged_sentence
0,3,3,"['For', 'you', 'have', 'been', 'reborn', ',', ..."
1,4,4,"['Man', 'had', 'enough', 'to', 'worry', 'about..."
2,6,6,"['He', 'trudged', 'on', ',', 'his', 'aching', ..."
3,12,12,"['``', 'But', 'I', 'believe', 'if', 'people', ..."
4,15,15,"['then', ',', 'to', 'better', 'purpose', ',', ..."
...,...,...,...
3995,9990,9990,"['The', 'way', 'MacArthur', 'said', 'his', 'li..."
3996,9991,9991,"['urged', 'the', 'reconvention', 'of', 'the', ..."
3997,9994,9994,"['A', 'prospective', 'industry', 'also', 'may'..."
3998,9997,9997,"['Mr.', 'McCormack', '.']"


In [107]:
test_data = []
for index, row in tqdm(test.iterrows()):
    test_data.append(ast.literal_eval(row['untagged_sentence']))

4000it [00:00, 15776.10it/s]


In [157]:
y = x
#x = ["start"] + x + ["end"]
vocab = set(vocab)

In [207]:
#Actual MEMM
all_predictions = []
for i in range(len(test_data)):
    memm = [[0 for _ in range(len(test_data[i])+1)] for _ in range(50)]
    backpointer = [[0 for _ in range(len(test_data[i])+1)] for _ in range(50)]
    if test_data[i][0] not in vocab:
            memm[1][0] = 1
            #backpointer[1][0] = 1
    else:
        for j in range(49):
            #memm[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[data[i][0][0]][y[j]]
            curr_word = vocab_dict[test_data[i][0]]
            curr_tag = tags_to_index[x[j]]
            prev_tag = 0
            memm[j][0] = matrix[curr_word][curr_tag][prev_tag]

    for k in range(1,len(test_data[i])):
        if test_data[i][k] not in vocab:
            memm[1][k] = 1
            backpointer[1][k] = 1
            continue
            
        for j in range(49):
            temp1 = []
            for l in range(49):
                #temp = path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]]
                curr_word = vocab_dict[test_data[i][k]]
                curr_tag = tags_to_index[x[j]]
                prev_tag = tags_to_index[x[l]]
                temp = memm[l][k-1] * matrix[curr_word][curr_tag][prev_tag]
                temp1.append(temp)
            memm[j][k] = np.max(np.array(temp1))
            backpointer[j][k] = np.argmax(np.array(temp1))
            
    temp = []
    flg = True
    for j in range(49):
        if test_data[i][len(test_data[i])-1] not in vocab:
            flg = False
            break
            
        #temp.append(path_prob_mat[j][len(data[i])-1] * trans_prob_mat['final'][y[j]])
        curr_word = vocab_dict[test_data[i][len(test_data[i])-1]]
        curr_tag = 49
        prev_tag = tags_to_index[x[j]]
        temp.append(memm[j][len(test_data[i])-1] * matrix[curr_word][curr_tag][prev_tag])
    
    if flg:
        memm[49][len(test_data[i])] = np.max(np.array(temp))
        backpointer[49][len(test_data[i])] = np.argmax(np.array(temp))
    else:
        memm[49][len(test_data[i])] = 1 #because i wanna assign probability 1 for "NN"
        backpointer[49][len(test_data[i])] = 1 #because the index for "NN" is 1
    
    preds = []
    temp = backpointer[49][len(test_data[i])]
    preds.append(x[temp])
    for j in range(len(test_data[i])-1,0,-1):
        preds.append(x[backpointer[temp][j]])
        temp = backpointer[temp][j]
    preds.reverse()
    all_predictions.append(preds)
print("Execution done")

Execution done


In [204]:
#Actual MEMM testing
all_predictions = []
for i in range(7,8):
    memm = [[0 for _ in range(len(data[i])+1)] for _ in range(50)]
    backpointer = [[0 for _ in range(len(data[i])+1)] for _ in range(50)]
    if data[i][0][0] not in vocab:
            memm[1][0] = 1
            #backpointer[1][0] = 1
    else:
        for j in range(49):
            #memm[j][0] = trans_prob_mat[y[j]]['start'] * emiss_prob_mat[data[i][0][0]][y[j]]
            curr_word = vocab_dict[data[i][0][0]]
            curr_tag = tags_to_index[data[i][0][1]]
            prev_tag = 0
            memm[j][0] = matrix[curr_word][curr_tag][prev_tag]

    for k in range(1,len(data[i])):
        if data[i][k][0] not in vocab:
            path_prob_mat[1][k] = 1
            backpointer[1][k] = 1
            continue
            
        for j in range(49):
            temp1 = []
            for l in range(49):
                #temp = path_prob_mat[l][k-1] * trans_prob_mat[y[j]][y[l]]
                curr_word = vocab_dict[data[i][k][0]]
                curr_tag = tags_to_index[data[i][k][1]]
                prev_tag = tags_to_index[x[l]]
                temp = memm[l][k-1] * matrix[curr_word][curr_tag][prev_tag]
                temp1.append(temp)
            memm[j][k] = np.max(np.array(temp1))
            backpointer[j][k] = np.argmax(np.array(temp1))
            
    temp = []
    for j in range(49):
        #temp.append(path_prob_mat[j][len(data[i])-1] * trans_prob_mat['final'][y[j]])
        curr_word = vocab_dict[data[i][len(data[i])-1][0]]
        curr_tag = 49
        prev_tag = tags_to_index[x[j]]
        temp.append(memm[j][len(data[i])-1] * matrix[curr_word][curr_tag][prev_tag])
    memm[49][len(data[i])] = np.max(np.array(temp))
    backpointer[49][len(data[i])] = np.argmax(np.array(temp))
    
    preds = []
    temp = backpointer[49][len(data[i])]
    preds.append(x[temp])
    for j in range(len(data[i])-1,0,-1):
        preds.append(x[backpointer[temp][j]])
        temp = backpointer[temp][j]
    preds.reverse()
    all_predictions.append(preds)

In [205]:
k = 7
cunt = 0
als = []
for i in range(len(data[k])):
    if preds[i] != data[k][i][1]:
        cunt = cunt + 1
        als.append(i)
for i in range(len(als)):
    print(data[k][als[i]])
    print(preds[als[i]])
cunt

('However', 'WR')
NN
(',', ',')
IN
('jury', 'NN')
PP
('it', 'PP')
NP
('believes', 'VB')
AT
('``', '``')
IN
('these', 'DT')
IN
('two', 'CD')
JJ
('be', 'BE')
AT
('achieve', 'VB')
AT
('and', 'CC')
TO
('reduce', 'VB')
IN
('of', 'IN')
AT
("''", "''")
AT
('.', '.')
AT


15

In [208]:
my_list = []
for i in range(len(all_predictions)):
    my_listt = []
    for j in range(len(all_predictions[i])):
        my_listt.append(tuple([test_data[i][j] , all_predictions[i][j]]))
    my_list.append(my_listt)

In [209]:
predictions = pd.DataFrame(columns = ["id" , "tagged_sentence"])
predictions['id'] = test['id']
predictions['tagged_sentence'] = my_list
predictions

,id,tagged_sentence
0,3,"[(For, AT), (you, AT), (have, AT), (been, AT),..."
1,4,"[(Man, AT), (had, AT), (enough, AT), (to, AT),..."
2,6,"[(He, AT), (trudged, AT), (on, AT), (,, AT), (..."
3,12,"[(``, AT), (But, AT), (I, AT), (believe, AT), ..."
4,15,"[(then, RB), (,, ,), (to, IN), (better, JJ), (..."
...,...,...
3995,9990,"[(The, AT), (way, NN), (MacArthur, NN), (said,..."
3996,9991,"[(urged, AT), (the, NN), (reconvention, NN), (..."
3997,9994,"[(A, AT), (prospective, JJ), (industry, NN), (..."
3998,9997,"[(Mr., AT), (McCormack, AT), (., AT)]"


In [210]:
predictions.to_csv("submission_10.csv" , index = False)